In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch import sigmoid

In [2]:
n_iters = 3000
n_epochs = 3 
batch_size_train = 32
batch_size_test = 1000
learning_rate = 0.01
input_dim = 784
output_dim = 10

In [3]:
#Normalized mean value of the dataset : 0.1307
#Normalized standart deviation of the dataset : 0.3081
train_loader = torch.utils.data.DataLoader(
  dsets.MNIST('/files/', train=True, download=True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                              (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=False)

test_loader = torch.utils.data.DataLoader(
  dsets.MNIST('/files/', train=False,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                               (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz to /files/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz to /files/MNIST/raw



In [4]:
class Logistic_Regression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Logistic_Regression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [5]:
model = Logistic_Regression(input_dim, output_dim)

In [6]:
criterion = torch.nn.CrossEntropyLoss()

In [7]:
def log_reg_select(optimizer):
  iter = 0
  for epoch in range(int(n_epochs)):
      for i, (images, labels) in enumerate(train_loader):
          images = Variable(images.view(-1, 28 * 28))
          labels = Variable(labels)

          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          iter+=1

          if iter%500==0:
              correct = 0
              total = 0
              for images, labels in train_loader:
                  images = Variable(images.view(-1, 28*28))
                  outputs = model(images)
                  _, predicted = torch.max(outputs.data, 1)
                  total+= labels.size(0)
                  correct+= (predicted == labels).sum()
              accuracy = 100 * correct/total
              print("Iteration: {}. Train_Loss: {}. Train_Accuracy: {}.".format(iter, loss.item(), accuracy))

          if iter%500==0:
              correct = 0
              total = 0
              for images, labels in test_loader:
                  images = Variable(images.view(-1, 28*28))
                  outputs = model(images)
                  _, predicted = torch.max(outputs.data, 1)
                  total+= labels.size(0)
                  correct+= (predicted == labels).sum()
              accuracy = 100 * correct/total
              print("Iteration: {}. Test_Loss: {}. Test_Accuracy: {}.".format(iter, loss.item(), accuracy))
  return

#Optimization without regularization 


In [8]:
optimizer_wo_reg = torch.optim.SGD(model.parameters(), lr=learning_rate)
log_reg_select(optimizer_wo_reg)

Iteration: 500. Train_Loss: 0.5494651198387146. Train_Accuracy: 88.93333435058594.
Iteration: 500. Test_Loss: 0.5494651198387146. Test_Accuracy: 89.66999816894531.
Iteration: 1000. Train_Loss: 0.2548758089542389. Train_Accuracy: 90.03333282470703.
Iteration: 1000. Test_Loss: 0.2548758089542389. Test_Accuracy: 90.62999725341797.
Iteration: 1500. Train_Loss: 0.1609133929014206. Train_Accuracy: 90.44499969482422.
Iteration: 1500. Test_Loss: 0.1609133929014206. Test_Accuracy: 90.75.
Iteration: 2000. Train_Loss: 0.23448042571544647. Train_Accuracy: 90.77999877929688.
Iteration: 2000. Test_Loss: 0.23448042571544647. Test_Accuracy: 91.1500015258789.
Iteration: 2500. Train_Loss: 0.2288506180047989. Train_Accuracy: 90.90166473388672.
Iteration: 2500. Test_Loss: 0.2288506180047989. Test_Accuracy: 91.37000274658203.
Iteration: 3000. Train_Loss: 0.3867492377758026. Train_Accuracy: 91.37999725341797.
Iteration: 3000. Test_Loss: 0.3867492377758026. Test_Accuracy: 91.62000274658203.
Iteration: 3500. 

#Optimization with L2 regularization

In [9]:
optimizer_w_reg = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.01)
log_reg_select(optimizer_w_reg)

Iteration: 500. Train_Loss: 0.46627068519592285. Train_Accuracy: 91.97333526611328.
Iteration: 500. Test_Loss: 0.46627068519592285. Test_Accuracy: 91.9000015258789.
Iteration: 1000. Train_Loss: 0.18492081761360168. Train_Accuracy: 91.94999694824219.
Iteration: 1000. Test_Loss: 0.18492081761360168. Test_Accuracy: 91.88999938964844.
Iteration: 1500. Train_Loss: 0.1294461190700531. Train_Accuracy: 91.86833190917969.
Iteration: 1500. Test_Loss: 0.1294461190700531. Test_Accuracy: 91.83999633789062.
Iteration: 2000. Train_Loss: 0.23093579709529877. Train_Accuracy: 91.90499877929688.
Iteration: 2000. Test_Loss: 0.23093579709529877. Test_Accuracy: 92.05000305175781.
Iteration: 2500. Train_Loss: 0.20476627349853516. Train_Accuracy: 91.7316665649414.
Iteration: 2500. Test_Loss: 0.20476627349853516. Test_Accuracy: 91.94999694824219.
Iteration: 3000. Train_Loss: 0.33040472865104675. Train_Accuracy: 91.98500061035156.
Iteration: 3000. Test_Loss: 0.33040472865104675. Test_Accuracy: 91.86000061035156